# Submission notebook for 2nd place solution in [UM - Game-Playing Strength of MCTS Variants](https://www.kaggle.com/competitions/um-game-playing-strength-of-mcts-variants)

Link to description: [2nd place solution](https://www.kaggle.com/competitions/um-game-playing-strength-of-mcts-variants/discussion/549718)

Link to model dataset, with instructions on how to reproduce the model binaries for the 2nd place solution (offline training): [um-gpsomctsv-2nd-place-solution](https://www.kaggle.com/datasets/fredrikupmark/um-gpsomctsv-2nd-place-solution)

The variable "original_CV_optimized_prop" below selects the desired blend between the two final submissions.

A value of 0 reproduces the predictions of the 2nd place winning, more Public Leaderboard aligned, submission.
This submission scored 0.41996 on the private test set and reached nr 2 on the private leaderboard.

A value of 1 reproduces the predictions of the second selected, more conservative, Cross Validation (CV) optimized submission.
This scored 0.42324 on the private test set. That is, this submission would have reached nr 6 on the Private Leaderboard (probably with Public Leaderboard aligned submissions ahead of it). Since this second submission was optimized to the larger CV data-set, it is possible (but definitely not certain) that blending values closer to 1 could generate predictions that generalize better outside the competition data.


In [ ]:
original_CV_optimized_prop = 0  #0 => fully trust LB, 1 => fully trust CV

In [ ]:
model_sets_to_use = list(range(6)) #Should be set to number of used seeds (i.e. 6 for the 2nd place solution)

In [ ]:
nr_of_gpus = 1 + (len(model_sets_to_use)>4)

In [ ]:
data_paths = {}
gpu_by_set = {}
neural_model_paths = {}
data_paths[0] = f'/kaggle/input/um-gpsomctsv-2nd-place-solution/UM - Game-Playing Strength of MCTS Variants - 2nd place solution/models/models_SEED_{0}/'
for set_nr in model_sets_to_use:
    data_paths[set_nr] = f'/kaggle/input/um-gpsomctsv-2nd-place-solution/UM - Game-Playing Strength of MCTS Variants - 2nd place solution/models/models_SEED_{set_nr}/'
    gpu_by_set[set_nr] = set_nr%nr_of_gpus

    neural_model_paths[set_nr] = f'/kaggle/input/um-gpsomctsv-2nd-place-solution/UM - Game-Playing Strength of MCTS Variants - 2nd place solution/models/models_SEED_{set_nr}/TENSORRT_PRE_MODEL'

In [ ]:
import os

import polars as pl

import numpy as np

import copy

import dill as pickle

import re


In [ ]:
import kaggle_evaluation.mcts_inference_server

In [ ]:
import lightgbm as lgb
print(lgb.__version__)

In [ ]:
import catboost
from catboost import CatBoostRegressor
catboost.__version__

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow import saved_model

In [ ]:
def split_agent_fields(df, agent_nr):
    df = df.with_columns(

        pl.col(f'agent{agent_nr}')

        .str.splitn('-', len(agent_fields))

        .struct.rename_fields(agent_fields)

        .alias("fields")

    ).unnest("fields")
    return df

In [ ]:
def process_agent_fields(df):
    for agent_nr in [1,2]:
        df = split_agent_fields(df, agent_nr)

        for agent_field in agent_fields_to_init_dict:
            for variable_sub_name in agent_fields_to_init_dict[agent_field]:
                df = df.with_columns( pl.lit(df[agent_field]==variable_sub_name).alias(f'AG{agent_nr}_{agent_field}_{variable_sub_name}') )

        df = df.drop(agent_fields)
        
    return df

In [ ]:
def cap_to_target_range(preds):
    return np.maximum(-1, np.minimum(1, preds))

Thx to [https://www.kaggle.com/code/yunsuxiaozi/mcts-starter](https://www.kaggle.com/code/yunsuxiaozi/mcts-starter) for the idea to calculate text scores!

In [ ]:
def ARI_McAlpine_EFLAW_CLRI(txt):
    characters=len(txt)
    words=len(re.split(' |\\n|\\.|\\?|\\!|\,',txt))
    sentence=len(re.split('\\.|\\?|\\!',txt))
    ari_score=4.71*(characters/words)+0.5*(words/sentence)-21.43
    mcalpine_eflaw_score=(words+sentence*words)/sentence
    L=100*characters/words
    S=100*sentence/words
    clri_score=0.0588*L-0.296*S-15.8
    return ari_score, mcalpine_eflaw_score, clri_score

In [ ]:
def clean_pd_col(pd_col):
    pd_col=pd_col.fillna("nan")
    pd_col=pd_col.apply(lambda x:x.lower())
    ps='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    for p in ps:
        pd_col=pd_col.apply(lambda x:x.replace(p,' '))
    return np.array(pd_col)

In [ ]:
def drop_gamename(rule):
    rule=rule[len('(game "'):]
    for i in range(len(rule)):
        if rule[i]=='"':
            return rule[i+1:]

In [ ]:
def get_player(rule):
    player=''
    stack=[]
    for i in range(len(rule)):
        player+=rule[i]
        if rule[i] in ['(','{']:
            stack.append(rule[i])
        elif rule[i] in [')','}']:
            stack=stack[:-1]
            if len(stack)==0:
                return player

In [ ]:
def get_ARI_McAlpine_EFLAW_CLRI_col_scores(tmp_df, col_name, target_str_name):
    cols_to_add = []
    ARI_McAlpine_EFLAW_CLRI_values = tmp_df[col_name].to_pandas().apply(lambda x:ARI_McAlpine_EFLAW_CLRI(x))
    ARI_McAlpine_EFLAW_CLRI_values = np.transpose(list(ARI_McAlpine_EFLAW_CLRI_values))
    for iter_nr, calc_score in enumerate(['_ARI', '_McAlpine_EFLAW', '_CLRI']):
        tmp_col_name = target_str_name+calc_score
        cols_to_add.append(tmp_col_name)
        tmp_df = tmp_df.with_columns( pl.lit( np.array( ARI_McAlpine_EFLAW_CLRI_values[iter_nr] ) ).alias(tmp_col_name) )    
        
    return tmp_df, cols_to_add

In [ ]:
def LudRules(df, group_col, new_rules_col):
    tmp_df = df.select(pl.col(group_col).value_counts()).unnest(group_col)
    tmp_df = tmp_df.with_columns( pl.lit( np.array( tmp_df[group_col].to_pandas().apply(lambda x:drop_gamename(x)) ) ).alias(new_rules_col) )
    tmp_df = tmp_df.with_columns( pl.lit( np.array( tmp_df[new_rules_col].to_pandas().apply(lambda rule:get_player(rule)) ) ).alias('player') )
    tmp_df = tmp_df.with_columns( pl.lit( np.array( [rule[len(player):] for player,rule in zip(tmp_df['player'].to_pandas(),tmp_df[new_rules_col].to_pandas())] ) ).alias(new_rules_col) )

    tmp_df, cols_to_add = get_ARI_McAlpine_EFLAW_CLRI_col_scores(tmp_df, new_rules_col, group_col)

    df = tmp_df[[group_col, new_rules_col]+cols_to_add].join(
        other = df,
        on = group_col,
        how = 'inner',
        validate = '1:m',
    )
    
    return df

In [ ]:
def predict(test: pl.DataFrame, sample_sub: pl.DataFrame):
    # Replace this function with your inference code.
    # You can return either a Pandas or Polars dataframe, though Polars is recommended.
    # Each batch of predictions (except the very first) must be returned within 10 minutes of the batch features being provided.

    
    #load neural models (placed within predict function to comply with the server time limit)
    if len(neural_models)==0:
        for set_nr in model_sets_to_use:
            with tf.device(f'/GPU:{gpu_by_set[set_nr]}'):
                loaded_neural_models[set_nr] = saved_model.load(neural_model_paths[set_nr])
                neural_models[set_nr] = loaded_neural_models[set_nr].signatures['serving_default']
                
    
    group_col = 'LudRules'
    test = LudRules(test, group_col, 'new_rules')
    
    test = test.with_columns( pl.lit( test['new_rules'] ).alias('LudRules') )
    test = test.drop(['new_rules'])

    
    model_structure, agent_fields, agent_fields_to_init_dict, TARGET, FEATURES, group_FEATURES_cols = FEATURES_etc_by_set[0]

    
    for FEATURE in group_FEATURES_cols:
        test = test.with_columns( pl.lit(False).alias(FEATURE) )

    
    
    test_ext = copy.deepcopy(test)

    test = test.with_columns( pl.lit(0).alias('is_ext_flipped') )
    test_ext = test_ext.with_columns( pl.lit(1).alias('is_ext_flipped') )


    test_ext = test_ext.with_columns( pl.lit(test['agent2']).alias('agent1') )
    test_ext = test_ext.with_columns( pl.lit(test['agent1']).alias('agent2') )


    #flip with 1-value
    test_ext = test_ext.with_columns( pl.lit(1-test['AdvantageP1']).alias('AdvantageP1') )
    

    full_test_df = pl.concat([test, test_ext])
    full_test_df = process_agent_fields(full_test_df)

    len_test = len(test)

    
    n_threads = 2

    
    #NEURAL prep
    test_neural = full_test_df
    NRL_FEATURES_NORMALIZED = []
    for FEATURE in NRL_FEATURES:
        FEATURE_SHRT = FEATURE[4:]
        if FEATURE in NRL_BOOL_FEATURES:
            NRL_FEATURES_NORMALIZED.append(FEATURE_SHRT)
        else:
            NRL_FEATURES_NORMALIZED.append(FEATURE+'_NORMALIZED')
            #per feature
            normalized_feature = np.interp(test_neural[FEATURE_SHRT], ppf_tuples_COMPRESSED[FEATURE], norm_ppf_tuple_COMPRESSED)
            test_neural = test_neural.with_columns( pl.lit(normalized_feature).alias(FEATURE+'_NORMALIZED') )

    pred_data = np.array(test_neural[NRL_FEATURES_NORMALIZED]).astype('float32')    
    

    full_preds_overall = []
    full_preds_overall_PubliLeaderboardFitted = []
    
    equal_w_preds_split_by_set = []
    equal_w_preds_overall_by_set = []
    equal_w_preds_reversed_split_by_set = []
    equal_w_preds_reversed_overall_by_set = []


    
    
    #predictions from BOOSTING models
    
    #catboost overall, unchanged model for all sets - only predict from first:
    catboost_overall_pre_preds_essential = models_by_set[0]['catboost_essential']['overall'].predict(np.array(full_test_df[FEATURES+group_FEATURES_cols]), thread_count=n_threads)
    catboost_overall_pre_preds = models_by_set[0]['catboost']['overall'].predict(np.array(full_test_df[FEATURES]), thread_count=n_threads)
    
    for set_nr in model_sets_to_use:
        equal_w_preds_split_by_set.append([])
        equal_w_preds_overall_by_set.append([])
        equal_w_preds_reversed_split_by_set.append([])
        equal_w_preds_reversed_overall_by_set.append([])
        
        preds_just_original = np.zeros(len_test)
        preds = np.zeros(len_test)
        preds_reversed_agents = np.zeros(len_test)
        
        model_structure, agent_fields, agent_fields_to_init_dict, TARGET, FEATURES, group_FEATURES_cols = FEATURES_etc_by_set[set_nr]
    
        for essential_add in ['', '_essential']:
            if essential_add=='_essential':
                pred_features = np.array(full_test_df[FEATURES+group_FEATURES_cols])
            else:
                pred_features = np.array(full_test_df[FEATURES])
    
            if essential_add=='_essential':
                model_w_prop = model_structure['weighting_props']['essential']
            else:
                model_w_prop = (1-model_structure['weighting_props']['essential'])
    
            for model_name in model_structure['weighting_props']['model_types']:
                model_w = model_structure['weighting_props']['model_types'][model_name]*model_w_prop
    
                model_structure_name = model_name+essential_add
    
                for cv_split_or_overall_switch in ['cv_splits', 'overall']:
                    if cv_split_or_overall_switch == 'cv_splits':
                        model_w_preds = model_w*(1-model_structure['weighting_props']['overall_models'])/nr_of_cv_splits
    
                        for split_model_nr in range(nr_of_cv_splits):
                            if model_name == 'lgbm':
                                pre_preds = models_by_set[set_nr][model_structure_name][cv_split_or_overall_switch][split_model_nr].predict(pred_features)
                            elif model_name == 'catboost':
                                pre_preds = models_by_set[set_nr][model_structure_name][cv_split_or_overall_switch][split_model_nr].predict(pred_features, thread_count=n_threads)
    
                            pre_preds = cap_to_target_range( pre_preds )
    
                            preds += pre_preds[:len_test] * model_w_preds
                            preds_reversed_agents += pre_preds[len_test:] * model_w_preds

                            if split_model_nr==0:
                                equal_w_preds_split_by_set[-1].append(pre_preds[:len_test]/nr_of_cv_splits)
                                equal_w_preds_reversed_split_by_set[-1].append(pre_preds[len_test:]*-1/nr_of_cv_splits)
                            else:
                                equal_w_preds_split_by_set[-1][-1] += pre_preds[:len_test]/nr_of_cv_splits
                                equal_w_preds_reversed_split_by_set[-1][-1] += pre_preds[len_test:]*-1/nr_of_cv_splits
    
                    else: #i.e. overall model
                        model_w_preds = model_w*model_structure['weighting_props']['overall_models']
    
                        if model_name == 'lgbm':
                            pre_preds = models_by_set[set_nr][model_structure_name][cv_split_or_overall_switch].predict(pred_features)
                        elif model_name == 'catboost':
                            if essential_add=='_essential':
                                pre_preds = catboost_overall_pre_preds_essential
                            else:
                                pre_preds = catboost_overall_pre_preds
    
                        pre_preds = cap_to_target_range( pre_preds )
    
                        preds += pre_preds[:len_test] * model_w_preds
                        preds_reversed_agents += pre_preds[len_test:] * model_w_preds

                        equal_w_preds_overall_by_set[-1].append(pre_preds[:len_test])
                        equal_w_preds_reversed_overall_by_set[-1].append(pre_preds[len_test:]*-1)
    
        
        pred_arrays = {}
        pred_arrays['tmp_ens_preds'] = preds
        pred_arrays['tmp_ens_preds_reversed_agents'] = preds_reversed_agents
        pred_arrays['AdvantageP1'] = np.array(test['AdvantageP1'])


        
        #predictions from NEURAL models
        with tf.device(f'/GPU:{gpu_by_set[set_nr]}'):
            tmp_preds_NN = neural_models[set_nr](tf.convert_to_tensor(pred_data))['output_0'].numpy().T

        
        capped_preds = cap_to_target_range( tmp_preds_NN[0] )
        tmp_preds = capped_preds * model_structure['weighting_props']['overall_models']
        pred_arrays['preds_NEURAL'] = tmp_preds[:len_test]
        pred_arrays['preds_reversed_agents_NEURAL'] = tmp_preds[len_test:]
        equal_w_preds_split_by_set[-1].append(capped_preds[:len_test])
        equal_w_preds_reversed_split_by_set[-1].append(capped_preds[len_test:]*-1)
        
        capped_preds = cap_to_target_range( tmp_preds_NN[3] )
        tmp_preds = capped_preds * (1-model_structure['weighting_props']['overall_models'])
        pred_arrays['preds_NEURAL'] += tmp_preds[:len_test]
        pred_arrays['preds_reversed_agents_NEURAL'] += tmp_preds[len_test:]
        equal_w_preds_overall_by_set[-1].append(capped_preds[:len_test])
        equal_w_preds_reversed_overall_by_set[-1].append(capped_preds[len_test:]*-1)

        
        capped_preds = cap_to_target_range( tmp_preds_NN[1] )
        tmp_preds = capped_preds * model_structure['weighting_props']['overall_models']
        pred_arrays['preds_NEURAL_AE'] = tmp_preds[:len_test]
        pred_arrays['preds_reversed_agents_NEURAL_AE'] = tmp_preds[len_test:]
        equal_w_preds_split_by_set[-1].append(capped_preds[:len_test])
        equal_w_preds_reversed_split_by_set[-1].append(capped_preds[len_test:]*-1)
        
        capped_preds = cap_to_target_range( tmp_preds_NN[4] )
        tmp_preds = capped_preds * (1-model_structure['weighting_props']['overall_models'])
        pred_arrays['preds_NEURAL_AE'] += tmp_preds[:len_test]
        pred_arrays['preds_reversed_agents_NEURAL_AE'] += tmp_preds[len_test:]
        equal_w_preds_overall_by_set[-1].append(capped_preds[:len_test])
        equal_w_preds_reversed_overall_by_set[-1].append(capped_preds[len_test:]*-1)


        capped_preds = cap_to_target_range( tmp_preds_NN[2] )
        tmp_preds = capped_preds * model_structure['weighting_props']['overall_models']
        pred_arrays['preds_NEURAL_BIG'] = tmp_preds[:len_test]
        pred_arrays['preds_reversed_agents_NEURAL_BIG'] = tmp_preds[len_test:]
        equal_w_preds_split_by_set[-1].append(capped_preds[:len_test])
        equal_w_preds_reversed_split_by_set[-1].append(capped_preds[len_test:]*-1)
        
        capped_preds = cap_to_target_range( tmp_preds_NN[5] )
        tmp_preds = capped_preds * (1-model_structure['weighting_props']['overall_models'])
        pred_arrays['preds_NEURAL_BIG'] += tmp_preds[:len_test]
        pred_arrays['preds_reversed_agents_NEURAL_BIG'] += tmp_preds[len_test:]
        equal_w_preds_overall_by_set[-1].append(capped_preds[:len_test])
        equal_w_preds_reversed_overall_by_set[-1].append(capped_preds[len_test:]*-1)
        

        #w/o full set of reversed agents
        reg_params = model_structure['reg_params_neural']
    
        preds_final = np.zeros(len_test)
        for param in reg_params.keys():
            if param=='Intercept':
                preds_final += reg_params[param]
            else:
                preds_final += reg_params[param] * pred_arrays[param]
    
        preds_final = cap_to_target_range( preds_final )
        
        reg_params = model_structure['reg_params_final_stretch']
        preds_final_stretched = np.zeros(len_test)
        for param in reg_params.keys():
            if param=='Intercept':
                preds_final_stretched += reg_params[param]
            else:
                preds_final_stretched += reg_params[param] * preds_final

        preds_final_stretched = cap_to_target_range( preds_final_stretched )
        
        
        #with full set of reversed agents
        reg_params = model_structure['reg_params_neural_FULL']
    
        preds_final = np.zeros(len_test)
        for param in reg_params.keys():
            if param=='Intercept':
                preds_final += reg_params[param]
            else:
                preds_final += reg_params[param] * pred_arrays[param]
    
        preds_final = cap_to_target_range( preds_final )
        
        reg_params = model_structure['reg_params_final_stretch_FULL']
        preds_final_stretched_FULL = np.zeros(len_test)
        for param in reg_params.keys():
            if param=='Intercept':
                preds_final_stretched_FULL += reg_params[param]
            else:
                preds_final_stretched_FULL += reg_params[param] * preds_final
        
        preds_final_stretched_FULL = cap_to_target_range( preds_final_stretched_FULL )

        
        
        full_preds_overall.append( preds_final_stretched )
        
        full_set_of_reversed_agents_prop = -3.5 #(value fitted through public leaderboard)
        full_preds_overall_PubliLeaderboardFitted.append( (1-full_set_of_reversed_agents_prop)*preds_final_stretched + full_set_of_reversed_agents_prop*preds_final_stretched_FULL )


    #combine preds from all sets
    median_prop = .75
    full_preds_overall = (1-median_prop) * np.mean(full_preds_overall, axis=0) + median_prop * np.median(full_preds_overall, axis=0)
    full_preds_overall_PubliLeaderboardFitted = (1-median_prop) * np.mean(full_preds_overall_PubliLeaderboardFitted, axis=0) + median_prop * np.median(full_preds_overall_PubliLeaderboardFitted, axis=0)

    
    #equal model weights preds
    concatenated_eq_preds = np.concatenate( (equal_w_preds_split_by_set, equal_w_preds_overall_by_set, equal_w_preds_reversed_split_by_set, equal_w_preds_reversed_overall_by_set), axis=1)
    concatenated_eq_preds = np.concatenate( concatenated_eq_preds, axis=0)
    median_prop = .75
    full_preds_overall_equal_w = (1-median_prop) * np.mean( concatenated_eq_preds, axis=0) + median_prop * np.median( concatenated_eq_preds, axis=0)
    equal_w_prop = -.6 #(value fitted through public leaderboard)
    full_preds_overall_PubliLeaderboardFitted = (1-equal_w_prop)*full_preds_overall_PubliLeaderboardFitted + equal_w_prop*full_preds_overall_equal_w

    
    #final blend of CV and PL optimized preds
    full_preds_overall_final = original_CV_optimized_prop * full_preds_overall + (1-original_CV_optimized_prop) * full_preds_overall_PubliLeaderboardFitted

    return sample_sub.with_columns(pl.Series('utility_agent1', cap_to_target_range(full_preds_overall_final) ))

In [ ]:
with open(data_paths[0] + 'neural_objects.pkl', "rb") as f:
    NRL_BOOL_FEATURES, NRL_FEATURES, NRL_FEATURES_NORMALIZED, norm_ppf_tuple_COMPRESSED, ppf_tuples_COMPRESSED = pickle.load(f)

In [ ]:
with open(data_paths[0] + 'model_structure_etc.pkl', "rb") as f:
    model_structure, agent_fields, agent_fields_to_init_dict, TARGET, FEATURES, group_FEATURES_cols = pickle.load(f)   
    

In [ ]:
FEATURES_etc_by_set = {}
for set_nr in [0] + model_sets_to_use:
    with open(data_paths[set_nr] + 'model_structure_etc.pkl', "rb") as f:
        FEATURES_etc_by_set[set_nr] = pickle.load(f)   


In [ ]:
loaded_neural_models = {}
neural_models = {}

In [ ]:
nr_of_cv_splits = 5

In [ ]:
#load catboost and lgbm models

use_lgbm_early_stop = True
if use_lgbm_early_stop:
    early_stop_str_add = '_early_stop'
else:
    early_stop_str_add = ''
    
use_catboost_early_stop = True
    

models_by_set = {}
for set_nr in model_sets_to_use:
    models_by_set[set_nr] = {}
    for essential_add in ['', '_essential']:
        for model_name in model_structure['weighting_props']['model_types']:
            model_structure_name = model_name+essential_add
            models_by_set[set_nr][model_structure_name] = {}
            for cv_split_or_overall_switch in ['cv_splits', 'overall']:
                if cv_split_or_overall_switch=='cv_splits':
                    cat_cv_splits_best_iterations = []
                    models_by_set[set_nr][model_structure_name][cv_split_or_overall_switch] = []
                    for split_model_nr in range(nr_of_cv_splits):
                        if model_name == 'lgbm':
                            tmp_model = lgb.Booster(model_file=data_paths[set_nr] +f'{model_name}{essential_add}_sub_{split_model_nr}{early_stop_str_add}.txt')
                        elif model_name == 'catboost':
                            tmp_model = CatBoostRegressor()
                            tmp_model.load_model(data_paths[set_nr] + f'{model_name}{essential_add}_sub_{split_model_nr}.cbm')
                            if use_catboost_early_stop:
                                best_iteration = tmp_model.get_best_iteration()
                                print(f'caboost best_iteration split {split_model_nr}: {best_iteration}')
                                cat_cv_splits_best_iterations.append(best_iteration)
                                tmp_model.shrink(ntree_end=best_iteration)
                        models_by_set[set_nr][model_structure_name][cv_split_or_overall_switch].append( tmp_model )
                else: #i.e. overall model
                    if model_name == 'lgbm':
                        tmp_model = lgb.Booster(model_file=data_paths[set_nr]+f'{model_name}{essential_add}.txt')
                        models_by_set[set_nr][model_structure_name][cv_split_or_overall_switch] = tmp_model
                        
                    elif model_name == 'catboost' and (set_nr==0 or 0 not in model_sets_to_use):
                        tmp_model = CatBoostRegressor()
                        tmp_model.load_model(data_paths[0] + f'{model_name}{essential_add}.cbm')
                        if use_catboost_early_stop:
                            best_iteration = 1 + round(.5*np.mean(np.asarray(cat_cv_splits_best_iterations)**.5)**2 + .5*np.median(cat_cv_splits_best_iterations))
                            print(f'caboost "best_iteration" overall: {best_iteration}')
                            tmp_model.shrink(ntree_end=best_iteration)
                        if 0 not in models_by_set:
                            models_by_set[0] = {}
                        if model_structure_name not in models_by_set[0]:
                            models_by_set[0][model_structure_name] = {}
                        models_by_set[0][model_structure_name][cv_split_or_overall_switch] = tmp_model


In [ ]:
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )